In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] ='6'
os.environ['PYOPENGL_PLATFORM'] = 'osmesa'
import torch
from pose_pipeline.utils.jupyter import play,play_grid
from pose_pipeline.pipeline import BlurredVideo,LiftingPerson,LiftingMethod,TopDownPerson,Video,VideoInfo
import logging
import mimetypes
import time
from argparse import ArgumentParser

import cv2
import json_tricks as json
import mmcv
import mmengine
import numpy as np

[2024-07-09 12:02:45,959][INFO]: Connecting pfirouzabadi@127.0.0.1:3306
[2024-07-09 12:02:46,059][INFO]: Connected pfirouzabadi@127.0.0.1:3306


In [2]:
from multi_camera.datajoint.multi_camera_dj import PersonKeypointReconstruction,SingleCameraVideo, CalibratedRecording, MultiCameraRecording,PersonKeypointReconstructionMethod
from multi_camera.datajoint.sessions import Recording


In [3]:
torch.cuda.set_per_process_memory_fraction(0.5, 0)
torch.cuda.empty_cache()

In [4]:
torch.cuda.is_available()


True

# Resources

In [5]:
from mmpose.apis import inference_topdown, init_model
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples, split_instances
from mmpose.utils import adapt_mmdet_pipeline
import cv2
try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False


from mmcv.image import imread


/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


# Detection and estimation model

In [6]:
def init_detector_model(detection_cfg, detection_ckpt,device):
    # build detector
    detector = init_detector(
        detection_cfg, detection_ckpt, device=device)
    detector.cfg = adapt_mmdet_pipeline(detector.cfg)
    return detector
def init_pose_model(pose_model_cfg, pose_model_ckpt,device):
    pose_estimator = init_model(pose_model_cfg, pose_model_ckpt, device=device)
    pose_estimator.cfg.visualizer.radius = 3
    pose_estimator.cfg.visualizer.alpha = 0.7
    pose_estimator.cfg.visualizer.line_width = 1
    
    return pose_estimator

def init_visualizer(pose_estimator):
    # build visualizer
    visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)
    # the dataset_meta is loaded from the checkpoint and
    # then pass to the model in init_pose_estimator
    visualizer.set_dataset_meta(
        pose_estimator.dataset_meta)
    return visualizer

In [7]:

# alg_name = "Freihand"
# pose_model_cfg = './configs/hand_2d_keypoint/topdown_heatmap/freihand2d/td-hm_res50_8xb64-100e_freihand2d-224x224.py'
# pose_model_ckpt = 'https://download.openmmlab.com/mmpose/hand/resnet/res50_freihand_224x224-ff0799bc_20200914.pth'

# alg_name = "HRNet_dark"
# pose_model_cfg = "./configs/hand_2d_keypoint/topdown_heatmap/rhd2d/td-hm_hrnetv2-w18_dark-8xb64-210e_rhd2d-256x256.py"
# pose_model_ckpt = "https://download.openmmlab.com/mmpose/hand/dark/hrnetv2_w18_rhd2d_256x256_dark-4df3a347_20210330.pth"
# alg_name = "RTMPoseCOCO"
# pose_model_cfg = './configs/hand_2d_keypoint/rtmpose/coco_wholebody_hand/rtmpose-m_8xb32-210e_coco-wholebody-hand-256x256.py'
# pose_model_ckpt = 'https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-coco-wholebody-hand_pt-aic-coco_210e-256x256-99477206_20230228.pth'

alg_name = "RTMPoseHand5"
pose_model_cfg = '/home/pfirouzabadi/projects/Hand_Detection/hand_detection/wrappers/mmpose/configs/hand_2d_keypoint/rtmpose/hand5/rtmpose-m_8xb256-210e_hand5-256x256.py'
pose_model_ckpt = 'https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-hand5_pt-aic-coco_210e-256x256-74fb594_20230320.pth'
device = 'cuda'

detection_cfg = '/home/pfirouzabadi/projects/Hand_Detection/hand_detection/wrappers/mmpose/demo/mmdetection_cfg/rtmdet_nano_320-8xb32_hand.py'
detection_ckpt = 'https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_nano_8xb32-300e_hand-267f9c8f.pth'


In [8]:
vid_keys = (SingleCameraVideo & 'filename LIKE "m001%disk_2024%"').fetch('KEY')
# (Video & vid_keys[::8][5]).fetch('video')
# keys= (SingleCameraVideo & vid_keys[5::8]).fetch('KEY')
# keys = np.array(keys)
# (Video &  keys[[4,1,3]]).fetch('video')

SingleCameraVideo & vid_keys#[[4,1,3]]

recording_timestamps,camera_config_hash,video_project,filename,camera_name,frame_timestamps
2024-06-14 12:48:23,39932d9e29,HAND_TEST,m001_disk_20240614_124823.23015083,23015083,=BLOB=
2024-06-14 12:48:23,39932d9e29,HAND_TEST,m001_disk_20240614_124823.23015089,23015089,=BLOB=
2024-06-14 12:48:23,39932d9e29,HAND_TEST,m001_disk_20240614_124823.23106526,23106526,=BLOB=
2024-06-14 12:48:23,39932d9e29,HAND_TEST,m001_disk_20240614_124823.23106528,23106528,=BLOB=
2024-06-14 12:48:23,39932d9e29,HAND_TEST,m001_disk_20240614_124823.23106530,23106530,=BLOB=
2024-06-14 12:48:23,39932d9e29,HAND_TEST,m001_disk_20240614_124823.23106542,23106542,=BLOB=
2024-06-14 12:48:23,39932d9e29,HAND_TEST,m001_disk_20240614_124823.23106544,23106544,=BLOB=
2024-06-14 12:48:23,39932d9e29,HAND_TEST,m001_disk_20240614_124823.23336088,23336088,=BLOB=


In [50]:
arr = (VideoInfo & keys[[4,1,3]]).fetch('timestamps')
np.save('timestamps.npy',arr)

In [64]:
(VideoInfo.populate(keys[2]))

In [19]:

info_key = (VideoInfo & (SingleCameraVideo & vid_keys)).fetch("KEY")[0]
timestamps = (VideoInfo & info_key).fetch_timestamps()[:]


In [65]:
VideoInfo & keys[2]

video_project,filename,timestamps,delta_time,fps,height,width,num_frames
HAND_TEST,m001_disk_force_20240614_134221.23106542,=BLOB=,=BLOB=,29.08,1536,2048,1847


In [47]:
participant_idx = 0
participant_videos = ((PersonKeypointReconstruction & (Recording & 'participant_id="m002"') & 'top_down_method=2' & 'reconstruction_method=0').fetch('KEY')[participant_idx])
vid_keys= (TopDownPerson * MultiCameraRecording * SingleCameraVideo & participant_videos).fetch('KEY')

# Run Experiment

In [9]:
detector = init_detector_model(detection_cfg=detection_cfg,detection_ckpt=detection_ckpt,device='cpu')
pose_estimator = init_pose_model(pose_model_cfg=pose_model_cfg, pose_model_ckpt=pose_model_ckpt, device=device)
visualizer = init_visualizer(pose_estimator)

output_root = f'./visualizations/{alg_name}/'
if not os.path.exists(output_root):
    os.mkdir(output_root)

#RUN VIDEOS FROM SPECIFIC VIEWS
for vid_idx in range(len(vid_keys)):
    vid_file = (Video &vid_keys[vid_idx]).fetch1('video')
    vid_file_prefix=  ''.join((SingleCameraVideo & vid_keys[vid_idx]).fetch1('filename').split('_')[:2])
    output_file = f'{output_root}{vid_file_prefix}_camera_{vid_idx}.mp4'
    
    cap = cv2.VideoCapture(vid_file)
    frame_idx = 0
    video_writer = None
    pred_instances_list = []
    while cap.isOpened():
        success, frame = cap.read()
        frame_idx += 1

        if not success:
            break
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        det_result = inference_detector(detector, frame)
        pred_instance = det_result.pred_instances.cpu().numpy()
        bboxes = np.concatenate(
            (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
        
        bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                        pred_instance.scores > .2)]
        bboxes = bboxes[nms(bboxes, .2), :4]
        # bboxes[:,:2] -= 50
        # bboxes[:,-2:] += 50

        
        # predict keypoints
        pose_results = inference_topdown(pose_estimator, frame, bboxes)
        data_samples = merge_data_samples(pose_results)

        visualizer.add_datasample(
            'result',
            frame,
            data_sample=data_samples,
            draw_gt=False,
            draw_heatmap=True,
            draw_bbox=True,
            show_kpt_idx=True,
            # show=args.show,
            wait_time=0,
            kpt_thr=0.3)
        # topdown pose estimation
        pred_instances = data_samples.get('pred_instances', None)

        # save prediction results
        pred_instances_list.append(
            dict(
                frame_id=frame_idx,
                instances=split_instances(pred_instances)))

        # output videos
        frame_vis = visualizer.get_image()

        if video_writer is None:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            # the size of the image with visualization may vary
            # depending on the presence of heatmaps
            video_writer = cv2.VideoWriter(
                output_file,
                fourcc,
                25,  # saved fps
                (frame_vis.shape[1], frame_vis.shape[0]))
        if frame_idx%300 == 0:
            ress=cv2.imwrite(f'{output_file[:-4]}_frame{frame_idx}.jpeg',frame_vis)

        video_writer.write(frame_vis)

        # if args.show:
        #     # press ESC to exit
        #     if cv2.waitKey(5) & 0xFF == 27:
        #         break

        #     time.sleep(args.show_interval)

    if video_writer:
        video_writer.release()

    cap.release()
    os.remove(vid_file)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_nano_8xb32-300e_hand-267f9c8f.pth


/home/pfirouzabadi/projects/Hand_Detection/hand_detection/wrappers/mmpose/mmpose/datasets/datasets/utils.py:102: UserWarning: The metainfo config file "configs/_base_/datasets/onehand10k.py" does not exist. A matched config file "/home/pfirouzabadi/projects/Hand_Detection/hand_detection/wrappers/mmpose/mmpose/.mim/configs/_base_/datasets/onehand10k.py" will be used instead.
  warnings.warn(
/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/mmdet/apis/inference.py:108: UserWarning: palette does not exist, random is used by default. You can also set the palette to customize.
  warnings.warn(


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-hand5_pt-aic-coco_210e-256x256-74fb594_20230320.pth


/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/mmdet/models/layers/se_layer.py:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/mmdet/models/backbones/csp_darknet.py:118: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/mmdet/models/layers/se_layer.py:158: Fut

In [15]:
import glob
folder = "/home/pfirouzabadi/projects/Hand_Detection/data/Gloves/*.mp4"
paths = glob.glob(folder)
''.join(paths[0].split('/')[-1].split('.')[0].split('_')[:2])


'CyberGlove'

In [17]:
detector = init_detector_model(detection_cfg=detection_cfg,detection_ckpt=detection_ckpt,device='cpu')
pose_estimator = init_pose_model(pose_model_cfg=pose_model_cfg, pose_model_ckpt=pose_model_ckpt, device=device)
visualizer = init_visualizer(pose_estimator)
output_root = f'./visualizations/{alg_name}/Gloves/'
if not os.path.exists(output_root):
    os.mkdir(output_root)

for vid_file in paths:
#RUN VIDEOS FROM SPECIFIC VIEWS
# for vid_idx in [5]:
# vid_file = "/home/pfirouzabadi/projects/Hand_Detection/data/fabio_0702/m010_hand_asl_20240702_123301.23336088.mp4"
# vid_file_prefix=  'm010_hand_asl_20240702_123301'
    vid_file_prefix= ''.join(vid_file.split('/')[-1].split('.')[0].split('_')[:2])
    output_file = f'{output_root}{vid_file_prefix}.mp4'

    cap = cv2.VideoCapture(vid_file)
    frame_idx = 0
    video_writer = None
    pred_instances_list = []
    while cap.isOpened():
        success, frame = cap.read()
        frame_idx += 1

        if not success:
            break
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        det_result = inference_detector(detector, frame)
        pred_instance = det_result.pred_instances.cpu().numpy()
        bboxes = np.concatenate(
            (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
        
        bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                        pred_instance.scores > .2)]
        bboxes = bboxes[nms(bboxes, .2), :4]
        # bboxes[:,:2] -= 50
        # bboxes[:,-2:] += 50

        
        # predict keypoints
        pose_results = inference_topdown(pose_estimator, frame, bboxes)
        data_samples = merge_data_samples(pose_results)

        visualizer.add_datasample(
            'result',
            frame,
            data_sample=data_samples,
            draw_gt=False,
            draw_heatmap=True,
            draw_bbox=True,
            show_kpt_idx=True,
            # show=args.show,
            wait_time=0,
            kpt_thr=0.3)
        # topdown pose estimation
        pred_instances = data_samples.get('pred_instances', None)

        # save prediction results
        pred_instances_list.append(
            dict(
                frame_id=frame_idx,
                instances=split_instances(pred_instances)))

        # output videos
        frame_vis = visualizer.get_image()

        if video_writer is None:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            # the size of the image with visualization may vary
            # depending on the presence of heatmaps
            video_writer = cv2.VideoWriter(
                output_file,
                fourcc,
                25,  # saved fps
                (frame_vis.shape[1], frame_vis.shape[0]))
        if frame_idx%210 == 0:
            ress=cv2.imwrite(f'{output_file[:-4]}_frame{frame_idx}.jpeg',frame_vis)

        video_writer.write(frame_vis)

        # if args.show:
        #     # press ESC to exit
        #     if cv2.waitKey(5) & 0xFF == 27:
        #         break

        #     time.sleep(args.show_interval)

    if video_writer:
        video_writer.release()

    cap.release()
    # os.remove(vid_file)

07/09 12:08:17 - mmengine - WARNING - The current default scope "mmpose" is not "mmdet", `init_default_scope` will force set the currentdefault scope to "mmdet".
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_nano_8xb32-300e_hand-267f9c8f.pth
07/09 12:08:17 - mmengine - WARNING - The current default scope "mmdet" is not "mmpose", `init_default_scope` will force set the currentdefault scope to "mmpose".


/home/pfirouzabadi/projects/Hand_Detection/hand_detection/wrappers/mmpose/mmpose/datasets/datasets/utils.py:102: UserWarning: The metainfo config file "configs/_base_/datasets/onehand10k.py" does not exist. A matched config file "/home/pfirouzabadi/projects/Hand_Detection/hand_detection/wrappers/mmpose/mmpose/.mim/configs/_base_/datasets/onehand10k.py" will be used instead.
  warnings.warn(
/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/mmdet/apis/inference.py:108: UserWarning: palette does not exist, random is used by default. You can also set the palette to customize.
  warnings.warn(


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-hand5_pt-aic-coco_210e-256x256-74fb594_20230320.pth


/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmpose.visualization.local_visualizer.PoseLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(
/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/mmdet/models/layers/se_layer.py:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/mmdet/models/backbones/csp_darknet.py:118: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/home/pfirouzabadi/.conda/envs/mmlab2/lib/python3.11/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming rel

# Sample Code for pose estimation

In [4]:
# !wget https://github.com/open-mmlab/mmpose/blob/dev-1.x/configs/hand_2d_keypoint/topdown_heatmap/freihand2d/td-hm_res50_8xb64-100e_freihand2d-224x224.py
# Detection model: https://download.openmmlab.com/mmpose/mmdet_pretrained/cascade_rcnn_x101_64x4d_fpn_20e_onehand10k-dac19597_20201030.pth
#RTMDET : https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_nano_8xb32-300e_hand-267f9c8f.pth

In [31]:
from mmcv.image import imread

from mmpose.apis import inference_topdown, init_model,visualize
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples

model_cfg = 'configs/hand_2d_keypoint/topdown_heatmap/freihand2d/td-hm_res50_8xb64-100e_freihand2d-224x224.py'

ckpt = 'https://dd.openmmlab.com/mmpose/hand/resnet/res50_freihand_224x224-ff0799bc_20200914.pth'

device = 'cuda:4'

# init model
model = init_model(model_cfg, ckpt, device=device)

Loads checkpoint by http backend from path: https://dd.openmmlab.com/mmpose/hand/resnet/res50_freihand_224x224-ff0799bc_20200914.pth


RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [32]:
img_path = 'tests/data/onehand10k/784.jpg'

# inference on a single image
batch_results = inference_topdown(model, img_path)

NameError: name 'model' is not defined

In [16]:
pred_instances = batch_results[0].pred_instances

pred_instances.keypoints

array([[[143.14285, 593.1429 ],
        [191.14285, 497.14285],
        [239.14285, 428.57144],
        [300.85715, 394.2857 ],
        [390.     , 360.     ],
        [204.85715, 380.57144],
        [266.57144, 346.2857 ],
        [328.2857 , 339.42856],
        [396.85715, 339.42856],
        [232.28572, 414.85715],
        [294.     , 394.2857 ],
        [355.7143 , 394.2857 ],
        [410.57144, 401.14285],
        [246.     , 483.42856],
        [294.     , 462.85715],
        [307.7143 , 483.42856],
        [300.85715, 510.85715],
        [266.57144, 531.4286 ],
        [294.     , 517.7143 ],
        [287.14285, 531.4286 ],
        [266.57144, 545.1429 ]]], dtype=float32)

In [17]:
model.cfg.visualizer

{'type': 'PoseLocalVisualizer',
 'vis_backends': [{'type': 'LocalVisBackend'}],
 'name': 'visualizer'}

In [18]:
visualizer = VISUALIZERS.build(model.cfg.visualizer)
pred_instances = batch_results[0].pred_instances
keypoints = pred_instances.keypoints
keypoint_scores = pred_instances.keypoint_scores
img = imread(img_path, channel_order='rgb')
results = merge_data_samples(batch_results)
# metainfo = 'configs/_base_/datasets/onehand10k.py'
visualizer.add_datasample(
            'result',
            img,
            data_sample=results,
            # det_data_sample=data_samples_2d,
            draw_gt=False,
            draw_bbox=True,
            kpt_thr=0.3,
            draw_heatmap=True,
            # show_kpt_idx=args.show_kpt_idx,
            # skeleton_style=args.skeleton_style,
            show=False,out_file='a.jpg',
            wait_time=0)

array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [ 39,  27,  27],
        [ 35,  23,  23],
        ...,
        [132, 104,  65],
        [132, 104,  67],
        [133, 105,  68]],

       [[  0, 255,   0],
        [ 35,  23,  23],
        [ 32,  20,  20],
        ...,
        [129, 104,  64],
        [131, 105,  68],
        [132, 106,  69]],

       ...,

       [[  0, 255,   0],
        [165, 173, 150],
        [165, 173, 150],
        ...,
        [193, 194, 176],
        [195, 194, 176],
        [194, 193, 175]],

       [[  0, 255,   0],
        [161, 169, 146],
        [160, 168, 145],
        ...,
        [192, 193, 175],
        [194, 193, 175],
        [194, 193, 175]],

       [[  0, 255,   0],
        [155, 163, 140],
        [153, 161, 138],
        ...,
        [190, 191, 173],
        [193, 192, 174],
        [194, 193, 175]]

In [ ]:
# from mmpose.apis import visualize

# pred_instances = batch_results[0].pred_instances

# keypoints = pred_instances.keypoints
# keypoint_scores = pred_instances.keypoint_scores

# metainfo = 'configs/_base_/datasets/coco_wholebody_hand.py'#'config/_base_/datasets/crowdpose.py'
# # configs/_base_/datasets/coco_wholebody_openpose.py
# visualize(
#     img_path,
#     keypoints,
#     keypoint_scores,
#     metainfo=metainfo,
#     # show=True)

In [ ]:
# # merge results as a single data sample
# # results = merge_data_samples(batch_results)

# # build the visualizer
# visualizer = VISUALIZERS.build(model.cfg.visualizer)

# # set skeleton, colormap and joint connection rule
# # visualizer.set_dataset_meta(model.dataset_meta)

# img = imread(img_path, channel_order='rgb')

# # visualize the results
# visualizer.add_datasample(
#     'result',
#     img,
#     data_sample=batch_results,
#     show=True)